In [ ]:
import numpy
import gym
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

import numpy as np
from evostra import EvolutionStrategy
from evostra.models import FeedForwardNetwork

from matplotlib import animation, rc
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
import matplotlib as mpl
from tqdm import tqdm

%matplotlib notebook

import os
import json

import random

In [ ]:
directory = 'abstraction-and-reasoning-challenge\\training\\'

In [ ]:
filename = '0ca9ddb6.json'

In [ ]:
dir_list = os.listdir(directory)

In [ ]:
len(dir_list)

In [ ]:
f = open(directory+filename, "r")
data = f.readlines()
print(len(data))
data = data[0]

In [ ]:
type(data)

In [ ]:
res = json.loads(data) 

In [ ]:
class big_NN(nn.Module):
    def __init__(self, num_channels):
        super(big_NN, self).__init__()
        temp = num_channels*3
        self.fc1 = nn.Linear(temp, temp)
        self.fc2 = nn.Linear(temp, temp)
        self.fc3 = nn.Linear(temp, num_channels)
        self.fc4 = nn.Sigmoid()
        #self.fc4 = nn.LeakyReLU()
        
        self.m1 = nn.Dropout(p=.01)
        self.m2 = nn.Dropout(p=.01)
        
    def forward(self,x):
        #x1 = self.fc4(self.fc1(x))
        x2 = self.fc4(self.fc3(x))
        #x3 = self.fc4(self.fc3(x2))
        return x2

In [ ]:
class out_height_model(nn.Module):
    def __init__(self):
        super(out_height_model, self).__init__()
        self.fc1 = nn.Linear(1, 1,bias=True)
        
    def forward(self,x):
        out = self.fc1(x)
        return out

In [ ]:
class out_width_model(nn.Module):
    def __init__(self):
        super(out_width_model, self).__init__()
        self.fc1 = nn.Linear(1, 1,bias=True)
        
    def forward(self,x):
        out = self.fc1(x)
        return out

In [ ]:
def get_inp_state(trial_num,res,inp_channels):
    inp_set = [c_num for c_num in range(10)]
    
    height = len(res['train'][trial_num]['input']) 
    width = len(res['train'][trial_num]['input'][0])
    
    inp_list = res['train'][trial_num]['input']
    
    inp_state = torch.zeros((inp_channels,height,width))
    
    for row in range(height):
        for col in range(width):
            num = inp_list[row][col]
            index = inp_set.index(num)
            inp_state[index][row][col] = 1
            
    temp_alive = torch.zeros((1,height,width))
    for i in range(1,inp_channels):
        temp_alive = temp_alive + inp_state[i]
        
    inp_state[inp_channels-1] = temp_alive
            
    return inp_state,height,width

In [ ]:
def get_ran_init_state(i,res,inp_channels):
    
    inp_set = [i for i in range(10)]
    num_vals = 11
    
    height = len(res['train'][i]['output']) 
    width = len(res['train'][i]['output'][0])
    
    inp_list = res['train'][i]['output']
    
    inp_state = torch.rand((inp_channels,height,width))*.3
    
    for row in range(height):
        for col in range(width):
            num = inp_list[row][col]
            index = inp_set.index(num)
            inp_state[index][row][col] = 1
            
    temp_alive = torch.zeros((1,height,width))
    for i in range(1,inp_channels):
        temp_alive = temp_alive + inp_state[i]
        
    inp_state[inp_channels-1] = temp_alive
            
    return inp_state,height,width

In [ ]:
def get_out_state(i,res,out_channels):
    
    inp_set = [i for i in range(10)]
    
    height = len(res['train'][i]['output']) 
    width = len(res['train'][i]['output'][0])
    
    inp_list = res['train'][i]['output']
    
    inp_state = torch.zeros((out_channels,height,width))
    
    for row in range(height):
        for col in range(width):
            num = inp_list[row][col]
            index = inp_set.index(num)
            inp_state[index][row][col] = 1
       
    #get one-hot list of all colors that are 'on'
    temp_alive = torch.zeros((1,height,width))
    for i in range(1,out_channels):
        temp_alive = temp_alive + inp_state[i]
        
    inp_state[-1] = temp_alive
            
    return inp_state,height,width

In [ ]:
def perception(state,num_channels):
    sobel_x = torch.FloatTensor([[-1,0,1],[-2,0,2],[-1,0,1]])
    sobel_y = torch.transpose(sobel_x,0,1)
    
    sobel_x = sobel_x.view(1,1,3,3)
    sobel_y = sobel_y.view(1,1,3,3)
    
    big_state = F.pad(state,(1,1,1,1),"constant",0)
    
    grad_x = F.conv2d(big_state, sobel_x,stride=1,padding=0)
    grad_y = F.conv2d(big_state, sobel_y,stride=1,padding=0)
    temp_state = torch.cat((state,grad_x,grad_y),0)
    
    return temp_state

In [ ]:
def init_state(state_inp,num_channels,out_height,out_width,inp_height,inp_width,inp_channels):
    state = torch.zeros((num_channels,1,out_height,out_width))
    
    #pad state input
    height_dif = out_height - inp_height
    width_dif = out_width - inp_width
    state_inp = state_inp.view(inp_channels,1,inp_height,inp_width)
    state_inp = F.pad(state_inp,(0,width_dif,0,height_dif),"constant",0)
    return state

In [ ]:
def stochastic_update(state,state_update,inp_channels,out_height,out_width,out_channels):
    #blur_state = F.max_pool2d(state[inp_channels],kernel_size=3,stride=1,padding=1)
    #alive = torch.where((blur_state>.1),torch.ones((num_channels,1,out_height,out_width)),torch.zeros((num_channels,1,out_height,out_width)))
    #new_state = state + state_update*alive + torch.normal(0, .02, size=state.shape)
    new_state = state + state_update + torch.normal(0, .02, size=state.shape)
    
    #print("state: {}  state_update: {}".format(state.shape,state_update.shape))
    
    #original state inputs shouldn't change
    for i in range(inp_channels):
        new_state[i] = state[i]
        
    #multiply output channels by predicted 
    pred_sum_out_channels = torch.round(state[-1])
    #pred_sum_out_channels = state[-1]
    for i in range(num_channels-out_channels,num_channels-1):
        new_state[i] = state[i]*pred_sum_out_channels
    
    return new_state

In [ ]:
for i in range(1,3):
    print(i)

In [ ]:
def run_out_pred(res,file_num,epochs):
    
    height_model = out_height_model()
    width_model = out_width_model()
    
    height_optimizer = optim.Adam(height_model.parameters(), lr=0.01)
    width_optimizer = optim.Adam(width_model.parameters(), lr=0.01)
    
    mse = nn.MSELoss()
    
    for epoch in range(epochs):
        tot_loss1 = 0
        tot_loss2 = 0
        for trial in range(len(res['train'])):
            height_optimizer.zero_grad()
            height_model.zero_grad()

            width_optimizer.zero_grad()
            width_model.zero_grad()

            state_inp,inp_height,inp_width = get_inp_state(trial,res,inp_channels)
            state_out,out_height,out_width = get_out_state(trial,res,inp_channels)

            inp_height = torch.FloatTensor([inp_height])
            inp_width = torch.FloatTensor([inp_width])

            out_height = torch.FloatTensor([out_height])
            out_width = torch.FloatTensor([out_width])

            out_height_pred = height_model(inp_height)
            out_width_pred = width_model(inp_width)

            loss1 = mse(out_height_pred,out_height)
            loss2 = mse(out_width_pred,out_width)

            loss1.backward()
            loss2.backward()

            height_optimizer.step()
            width_optimizer.step()
            tot_loss1+=loss1
            tot_loss2+=loss2
        if(tot_loss1<.0001 and tot_loss2<.0001):
            break
            #print("True out height: {}  Pred out height: {}".format(out_height,out_height_pred))
            #print("True out width: {}  Pred out width: {}".format(out_width,out_width_pred))
        
    PATH1 = "models\\"+str(file_num)+"_height_model.pt"
    PATH2 = "models\\"+str(file_num)+"_width_model.pt"
    # Save
    torch.save(height_model, PATH1)
    torch.save(width_model, PATH2)
        
    return tot_loss1,tot_loss2

In [ ]:
def run(res,file_num):
    
    global steps, num_channels, inp_channels, tau, model, optimizer
    
    PATH1 = "models\\"+str(file_num)+"_height_model.pt"
    PATH2 = "models\\"+str(file_num)+"_width_model.pt"
    
    height_model = torch.load(PATH1)
    width_model = torch.load(PATH2)
    
    mse = nn.MSELoss()
    ce = nn.CrossEntropyLoss()
    
    loss_list = []
    tot_loss = 0
    for trial in range(len(res['train'])):
        
        model.zero_grad()
        optimizer.zero_grad()
        
        temp_choice = np.random.choice([0,1],1,p=[1.0,0.0])
        temp_dict = {0: get_inp_state,1: get_ran_init_state}
        
        state_inp,inp_height,inp_width = temp_dict[temp_choice[0]](trial,res,inp_channels)
        
        #get predicted output height
        inp_height_tensor = torch.FloatTensor([inp_height])
        out_height = torch.round(height_model(inp_height_tensor))
        out_height = int(out_height)
        
        #get predicted output width
        inp_width_tensor = torch.FloatTensor([inp_width])
        out_width = torch.round(width_model(inp_width_tensor))
        out_width = int(out_width)
        
        num_inp = out_height*out_width
        
        #run 
        state_inp = torch.FloatTensor(state_inp)
        temp_state = init_state(state_inp,num_channels,out_height,out_width,inp_height,inp_width,inp_channels)
        for step in range(steps):
            net_inp = perception(temp_state,num_channels-inp_channels)
            net_inp = torch.transpose(net_inp, 0, 3)
            net_inp = torch.transpose(net_inp, 1, 2)
            net_inp = net_inp.reshape((num_inp,num_channels*3))

            state_update = model(net_inp)

            state_update = state_update.view((out_width,out_height,1,num_channels))
            state_update = torch.transpose(state_update, 1, 2)
            state_update = torch.transpose(state_update, 0, 3)
            state_update = stochastic_update(temp_state,state_update,inp_channels,out_height,out_width,out_channels)
            temp_state = state_update
            
            #multiple predicted output by all output squares
            
        #get losses
        true_out_state,out_height,out_width = get_out_state(trial,res,out_channels)
        true_out_state = torch.FloatTensor(true_out_state)

        pred_out_state = temp_state[num_channels-out_channels:num_channels].view(out_channels,out_height,out_width)
        pred_out_state = F.softmax(pred_out_state,0)
        loss = mse(true_out_state[-1],pred_out_state[-1])
        #loss = mse(true_out_state,pred_out_state)
        #loss.backward(retain_graph=True)
        #optimizer.step()
        
        loss_list.append(loss)
        tot_loss+=loss
        
    temp_tot_loss = sum(loss_list)
    temp_tot_loss.backward(retain_graph=True)
    optimizer.step()
    
    loss_list = [round(float(temp_loss.detach()),3) for temp_loss in loss_list]
    return tot_loss,loss_list

In [ ]:
def playthrough(res,file_num):
    
    global steps, num_channels, inp_channels, tau, model, optimizer
    
    PATH1 = "models\\"+str(file_num)+"_height_model.pt"
    PATH2 = "models\\"+str(file_num)+"_width_model.pt"
    
    height_model = torch.load(PATH1)
    width_model = torch.load(PATH2)
    
    mse = nn.MSELoss()
    ce = nn.CrossEntropyLoss()
    
    loss_list = []
    tot_loss = 0
    pred_final_list = []
    for trial in range(len(res['train'])):
        
        temp_choice = np.random.choice([0,1],1,p=[1.0,0.0])
        temp_dict = {0: get_inp_state,1: get_ran_init_state}
        
        state_inp,inp_height,inp_width = temp_dict[temp_choice[0]](trial,res,inp_channels)
        
        #get predicted output height
        inp_height_tensor = torch.FloatTensor([inp_height])
        out_height = torch.round(height_model(inp_height_tensor))
        out_height = int(out_height)
        
        #get predicted output width
        inp_width_tensor = torch.FloatTensor([inp_width])
        out_width = torch.round(width_model(inp_width_tensor))
        out_width = int(out_width)
        
        num_inp = out_height*out_width
        
        #run 
        state_inp = torch.FloatTensor(state_inp)
        temp_state = init_state(state_inp,num_channels,out_height,out_width,inp_height,inp_width,inp_channels)
        for step in range(steps):
            net_inp = perception(temp_state,num_channels-inp_channels)
            net_inp = torch.transpose(net_inp, 0, 3)
            net_inp = torch.transpose(net_inp, 1, 2)
            net_inp = net_inp.reshape((num_inp,num_channels*3))

            state_update = model(net_inp)

            state_update = state_update.view((out_width,out_height,1,num_channels))
            state_update = torch.transpose(state_update, 1, 2)
            state_update = torch.transpose(state_update, 0, 3)
            state_update = stochastic_update(temp_state,state_update,inp_channels,out_height,out_width,out_channels)
            temp_state = state_update
            
        #get losses
        true_out_state,out_height,out_width = get_out_state(trial,res,out_channels)
        true_out_state = torch.FloatTensor(true_out_state)

        pred_out_state = temp_state[num_channels-out_channels:num_channels].view(out_channels,out_height,out_width)
        print(pred_out_state.shape)
        print(torch.round(pred_out_state[-1].detach()).numpy())
        pred_out_state = F.softmax(pred_out_state,0)
        #print(true_out_state.shape)
        #print(temp_state.shape)
        #print(pred_out_state.shape)
        loss = mse(true_out_state[-1],pred_out_state[-1])
        tot_loss+=loss

        #convert tensor to standard output
        org_shape = pred_out_state.shape
        
        #get list of symbols
        inp_set = [i for i in range(10)]
        
        #turn one-hot into symbols tensor
        
        pred_out_state = temp_state[1+num_channels-out_channels:num_channels].view(out_channels-1,out_height,out_width)
        pred_out_state = F.softmax(pred_out_state,0)
        pred_out_state = torch.transpose(pred_out_state, 0, 1)
        pred_out_state = torch.transpose(pred_out_state, 1, 2)
        #print(pred_out_state.shape)
        #print("{} vs {}".format(out_height,out_width))
        temp_final = torch.zeros((out_width,out_height))
        count1 = -1
        for row in pred_out_state:
            count1+=1
            count2 = -1
            for col in row:
                count2+=1
                num = torch.argmax(col)
                temp_final[count1][count2] = inp_set[num]  
        pred_final = torch.transpose(temp_final, 0, 1)
        true_final = torch.FloatTensor(res['train'][trial]['output'])
        if(torch.all(torch.eq(pred_final, true_final))):
            print("------------True-------------")
        else:
            print("False:  {}   {}  {} -------".format(pred_final.shape,true_final.shape,pred_final.shape==true_final.shape))
        
        print("Loss: {}".format(loss))
        pred_final_list.append(pred_final)
    return tot_loss,pred_final_list

In [ ]:
steps = 10
state_pred_epochs = 3000
out_pred_epochs = 3000

In [ ]:
dir_list = os.listdir(directory)
directory = 'abstraction-and-reasoning-challenge\\training\\'

In [ ]:
file_num = -1
for filename in dir_list:
    file_num+=1

    if(file_num<4):
        continue
    
    f = open(directory+filename, "r")
    data = f.readlines()
    data = data[0]
    res = json.loads(data)
    
    #define channel sizes
    inp_channels = 11
    inf_channels = 4
    out_channels = 11
    num_channels = inp_channels + inf_channels + out_channels
    
    #define models
    
        
    model = big_NN(num_channels)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    tau = .01
    
    print("--------------------New File: {}--------------------".format(file_num))
    print("--------output length model--------")
    #train output length prediction model
    tot_loss1, tot_loss2 = run_out_pred(res,file_num,out_pred_epochs)
    print("loss1: {}  loss2: {}".format(tot_loss1,tot_loss2))
    while(tot_loss1>.0001 or tot_loss2>.0001):
        tot_loss1, tot_loss2 = run_out_pred(res,file_num,out_pred_epochs)
        print("loss1: {}  loss2: {}".format(tot_loss1,tot_loss2))
        
    print("--------state prediction model-----")
    
    PATH1 = "models\\"+str(file_num)+"_height_model.pt"
    PATH2 = "models\\"+str(file_num)+"_width_model.pt"

    # Load
    height_model = torch.load(PATH1)
    width_model = torch.load(PATH2)
    
    #train state prediction model
    for epoch in range(state_pred_epochs):
        tot_loss,loss_list = run(res,file_num)
        print("epoch {}  loss: {}: loss_list: {}".format(epoch,tot_loss,loss_list))
        if(tot_loss<.0005):
            break
    
    playthrough(res,file_num)
    
    PATH = "models\\"+str(file_num)+"_model.pth"
    torch.save(model, PATH)
    
    break

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
    
def plot_pictures(pictures, labels):
    fig, axs = plt.subplots(1, len(pictures), figsize=(3*len(pictures),32))
    for i, (pict, label) in enumerate(zip(pictures, labels)):
        axs[i].imshow(np.array(pict), cmap=cmap, norm=norm)
        axs[i].set_title(label)
    plt.show()
    

def plot_sample(sample, predict=None):
    if predict is None:
        plot_pictures([sample['input'], sample['output']], ['Input', 'Output'])
    else:
        plot_pictures([sample['input'], sample['output'], predict], ['Input', 'Output', 'Predict'])

In [ ]:
file_num = 1
temp_filename = os.listdir(directory)[file_num]
f = open(directory+filename, "r")
print(temp_filename)
data1 = f.readlines()
data1 = data1[0]
res1 = json.loads(data1)

In [ ]:
#PATH = "models\\"+str(file_num)+"_model.pth"
#model = torch.load(PATH)
loss,pred_final = playthrough(res1,file_num)

In [ ]:
sample_num = 0
plot_sample(res['train'][sample_num],pred_final[sample_num])

In [ ]:
print(pred_final[sample_num].shape)

In [ ]:
#need to convert this back to numbers
inp_state,height,width = get_inp_state(sample_num,res,inp_channels)

In [ ]:
inp_state.shape

In [ ]:
print(inp_state[-1].numpy())

In [ ]:
out_state,height,width = get_out_state(sample_num,res,inp_channels)

In [ ]:
out_state.shape

In [ ]:
print(out_state[-1].numpy())